## KoGPT2
 Day04-1104 ~ Day06-1106

* MODEL3 : SKT koGPT 2
 - Base Model(110M) 까지는 LMS 환경에서 사용 가능하다고 들어 적합한 사이즈의 모델을 서칭하여 선정
 - dataset 구성 : 최종  train 14000, val 3500, test 3500 참고 할것

 - [Huggingface_GPT2](https://huggingface.co/docs/transformers/model_doc/gpt2)
 - [SKT-koGPT2](https://github.com/SKT-AI/KoGPT2#model)
 - [GPT2_finetuning](https://hyyoka-ling-nlp.tistory.com/9)
 - [GPT2_finetuning_NonEng](https://www.philschmid.de/fine-tune-a-non-english-gpt-2-model-with-huggingface)
 - [GPT2 text summarization finetuning](https://towardsdatascience.com/text-summarization-with-gpt2-and-layer-ai-599625085d8e)

* 추상요약
 - [Abstractive Summarization](https://medium.com/search?q=abstractive+text+summarization)


 GPT2 모델 학습
  1. Data 로드
   * 데이터 형식 변경 : train/validation/ test 하나로 취합하여 로드  
   * 데이터 구성 : train/validation/test 형식에 맞추어 변경 후 데이터 다운로드)

2. GPT Tokenizer, Model 허깅페이스 로드 
 * 4단계.trainer에서 dimension mismatch 오류가 지속적으로 발생하여 학습불가 
    * 2단계.토크나이징 단계에 동적 패딩 적용하여 디멘젼 오류 해결

3. Load Train Config 
 * 5.test dataset evaluation단계 까지 완료 한 후 다시 3.Load Train Config 단계로 돌아와 파인튜닝하여 성능 향상 최종 목표!)

4. Trainer : 
 * traindatat 학습 예상 시간이 너무 길어서 학습 불가
    * max step size 설정하여 train data학습양과 시간 줄여서 학습 성공
 * validation dataset의 크기가 너무 커서 현재 trainer 완료 하지 못한 상태
    * 해결책 찾는중
    
5. Test evaluation 

In [ ]:
! pip uninstall -y transformers
! pip install transformers

In [ ]:
!pip uninstall -y datasets
!pip install datasets

# PROJECT : 뉴스 데이터셋 요약
* Task : summary

In [ ]:
import os
import numpy as np
from argparse import ArgumentParser
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification, AutoConfig
from dataclasses import asdict
from transformers.data.processors.utils import DataProcessor, InputExample, InputFeatures

## (1) Dataset Load

>Dataset dictionary
* train dataset, validation dataset, test dataset으로 구성
* Dataset은 ‘sentence1’, ‘sentence2’, ‘label’, ‘idx’(인덱스)로 구성 

 Huggingface datasets를 사용하면 DataProcessor를 사용할 필요없이 바로 사용가능하게 구성되어 있습니다.

In [ ]:
# import datasets
# from datasets import load_dataset

# huggingface_mrpc_dataset = load_dataset('glue', 'mrpc')
# print(huggingface_mrpc_dataset)

>News Dataset dictionary
* train dataset, validation dataset 으로 구성
* Dataset은 ‘input’, target'로 구성 

In [5]:
# Load a CSV file
from datasets import load_dataset, Dataset
# train_Sum1 = load_dataset('csv', data_files='/content/drive/MyDrive/Jiwon/Data/train_Sum1.csv')
# val_Sum1 = load_dataset('csv', data_files='/content/drive/MyDrive/Jiwon/Data/val_Sum1.csv')

In [ ]:
# %cd AIFFELTHON_DATA

In [ ]:
# !pwd


* 데이터 로드할때 형식 맞춰주기 
```
data_files = {"train": "/content/drive/MyDrive/Jiwon/Data/train_Sum1.csv", "test": "/content/drive/MyDrive/Jiwon/Data/val_Sum1.csv"}
dataset = load_dataset('csv', data_files=data_files)
```



In [6]:
# root = '/content/drive/MyDrive/Jiwon/Data/'
root = '/aiffel/aiffel/Jiwon/AIFFELTHON_DATA/'

data_files = {
    "train" : root + "train_Sum1_.csv",
    "val"   : root + "val_Sum1.csv",
    "test"  : root + "test_Sum1.csv"
    }
dataset = load_dataset('csv', data_files=data_files)

Using custom data configuration default-e14cee014c714360


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /aiffel/.cache/huggingface/datasets/csv/default-e14cee014c714360/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset

In [ ]:
dataset['train'][1]

In [ ]:
# import pandas as pd

# train_Sum1 = pd.read_csv("/content/drive/MyDrive/Jiwon/Data/train_Sum1.csv")
# val_Sum1 = pd.read_csv("/content/drive/MyDrive/Jiwon/Data/val_Sum1.csv")

In [ ]:
# train_Sum1.head()

In [ ]:
# val_Sum1.head()

## (2) Tokenizer, Model

> Model, Tokenizer 생성
* Huggingface- Auto Class
 - Auto class는 from_pretrained 메소드를 이용해 pretrained model의 경로 혹은 이름만 안다면 자동으로 생성하는 방법
 - AutoTokenizer, AutoModel기능 지원



model = AutoModelForCausalLM.from_pretrained

In [7]:
from transformers import BertTokenizerFast, GPT2LMHeadModel
tokenizer_gpt3 = BertTokenizerFast.from_pretrained("kykim/gpt3-kor-small_based_on_gpt2")
input_ids = tokenizer_gpt3.encode("text to tokenize")[1:]  # remove cls token
        
model_gpt3 = GPT2LMHeadModel.from_pretrained("kykim/gpt3-kor-small_based_on_gpt2")

Downloading:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/344k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'BertTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

In [8]:
tokenizer_gpt3.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o")

['안녕하세요',
 '.',
 '한국어',
 'g',
 '##pt',
 '-',
 '2',
 '입니다',
 '.',
 '[UNK]',
 ':',
 ')',
 'l',
 '^',
 'o']

In [ ]:
import torch
from transformers import GPT2LMHeadModel

# model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
text = '근육이 커지기 위해서는'
input_ids = tokenizer_gpt3.encode(text, return_tensors='pt')
gen_ids = model_gpt3.generate(input_ids,
                          max_length=128,
                          repetition_penalty=2.0,
                          pad_token_id=tokenizer_gpt3.pad_token_id,
                          eos_token_id=tokenizer_gpt3.eos_token_id,
                          bos_token_id=tokenizer_gpt3.bos_token_id,
                          use_cache=True)
generated = tokenizer_gpt3.decode(gen_ids[0])
print(generated)

In [ ]:
# from transformers import GPT2TokenizerFast
# tokenizer = GPT2TokenizerFast.from_pretrained("skt/kogpt2-base-v2",
# bos_token='</s>', eos_token='</s>', unk_token='<unk>',
# pad_token='<pad>', mask_token='<mask>')

In [ ]:
# tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o")

In [ ]:
# import torch
# from transformers import GPT2LMHeadModel

# model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
# text = '근육이 커지기 위해서는'
# input_ids = tokenizer.encode(text, return_tensors='pt')
# gen_ids = model.generate(input_ids,
#                           max_length=128,
#                           repetition_penalty=2.0,
#                           pad_token_id=tokenizer.pad_token_id,
#                           eos_token_id=tokenizer.eos_token_id,
#                           bos_token_id=tokenizer.bos_token_id,
#                           use_cache=True)
# generated = tokenizer.decode(gen_ids[0])
# print(generated)

In [ ]:
gen_ids

> 토크나이징
* `transform()` 함수를 만들고 이전에 만들어두었던 Tokenizer에서 입력할 `dataset의 형태를 확인하고 바꿀 대상을 지정`해야 합니다.

In [ ]:
!pip list | grep transformers

In [59]:
def transform(data):
  return tokenizer_gpt3(
      text = data['input'],
      text_target = data['target'],
#       text_pair_target=data['target'],
      truncation = True,
      padding = 'max_length',
      return_token_type_ids = False,
      max_length=128
      )
  
examples = dataset['train'][:5]
examples_transformed = transform(examples)

print(examples)
print(examples_transformed)

{'input': ['4일 오후 4시 충남 아산시 신리초등학교 체육관 2층 입구.\n  손 소독과 발열 체크를 한 뒤 체육관 안으로 들어가라는 안내문이 붙었다.\n  마스크를 착용하지 않으면 출입이 통제됐다.\n  체육관으로 들어가자 왼쪽에 설치된 대형 칠판에 ‘사람 두 팔 간격 건강 거리 유지’라는 글이 크게 쓰여 있었다.\n  지난 3일 충남 아산지역에 폭우가 내리면서 온양천이 범람하자 신동·모종동 주민들이 급히 신리초로 대피했다.\n  신리초는 홍수와 지진 등 각종 재난사고 때 임시 주거시설로 지정된 곳이다.\n  3일 오후 5시까지 60여 명 수준이던 이재민은 오후 7시가 지나서는 100여 명을 넘어섰다.\n  아산시는 주민들이 한꺼번에 몰리자 수해와 더불어 신종 코로나바이러스 감염증(코로나19)이 확산할 것을 우려했다.\n  가장 먼저 주민들에게 마스크 착용을 의무화했다.\n  마스크를 쓰지 않으면 체육관 출입을 제한하고 체육관에 머무는 시간에도 밥을 먹거나 물을 마실 때를 제외하고 마스크를 벗지 못하게 했다.\n  이재민들은 집이나 농경지로 오갈 때도 출입 횟수에 관계없이 매번 발열 체크를 했다.\n  외부에서 다른 사람과 접촉할 가능성이 있어서다.\n  4일 오후 4시 30분쯤 체육관 안에는 주민 5명과 아산시청 공무원·자원봉사자 10여 명이 머물고 있었다.\n  주민들은 아산시가 제공한 매트리스 위에서 쉬거나 대화를 나눴다.\n  매트리스 1개마다 1명씩 배정됐다.\n  매트리스 간격은 2~3m가량으로 1개의 매트리스에 2명 이상이 앉지 못하도록 했다.\n  주민 간 접촉을 최소화하기 위해서다.\n  다른 대피소와 달리 신리초 체육관에는 소형 천막(텐트)이 지급되지 않았다.\n  이 때문에 충남도와 아산시 등 방역 당국이 바짝 긴장했다.\n  이곳에 머무르는 주민들은 대부분 70~80대 고령자로 코로나19에 노출될 경우 감염 위험이 높아서다.\n  아산시는 주민들이 식사를 할 때 밀접 접촉할 가능성이 크다고 판단, 모든 식사를 도시락으로 대체했다

In [58]:
examples_transformed.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [ ]:
# def transform(data):
#   return tokenizer(
#       text = data['input'],
#       text_target = data['target'],
#       truncation = True,
#       padding = 'max_length',
#       return_token_type_ids = False,
#       max_length=128
#       )
  
# examples = dataset['train'][:5]
# examples_transformed = transform(examples)

# print(examples)
# print(examples_transformed)

In [ ]:
# import numpy as np
# len(encoded_data['test']['input_ids'][0]), len(encoded_data['train']['input_ids'][0])

In [ ]:
# len(encoded_data['test']['labels'][0]), len(encoded_data['train']['labels'][0])

In [ ]:
# len(encoded_data['test']['attention_mask'][0]), len(encoded_data['train']['attention_mask'][0])

expected sequence of length 278 at dim 1 (got 281)

In [ ]:
# import seaborn as sns 

> `map():`
* Data dictionary에 있는 모든 데이터들이 한번에 토크나징 적용시킬 수 있습니다.
* 우리는 map을 사용해 토크나이징을 진행하기 때문에 batch를 적용해야 되므로 batched=True 설정

In [60]:
encoded_data = dataset.map(transform, batched=True)

  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/19 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [61]:
encoded_data

DatasetDict({
    train: Dataset({
        features: ['input', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 15250
    })
    val: Dataset({
        features: ['input', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 18300
    })
    test: Dataset({
        features: ['input', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3050
    })
})

## (3) Train/Evaluation과 Test

>Trainer
* 사용하기 위해서는 TrainingArguments를 통해 학습 관련 설정을 미리 지정

In [11]:
! pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     |████████████████████████████████| 72 kB 1.7 MB/s            


In [15]:
! pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=f04c96d915d85c69c9765936f38aa57f0659a661d6fe668dcf301d902abf8503
  Stored in directory: /aiffel/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge-score


In [12]:
import evaluate

In [ ]:
# def _rouge(guess, answers):
#     global rouge
#     """Compute ROUGE score between guess and *any* answers. Return the best."""
#     if rouge is None:
#         return None, None, None
#     evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l'], max_n=2)
#     try:
#         scores = [
#             evaluator.get_scores(normalize_answer(guess), normalize_answer(a))
#             for a in answers
#         ]
#     except LookupError:
#         warn_once(
#             'ROUGE requires nltk punkt tokenizer. Please run '
#             '`python -c "import nltk; nltk.download(\'punkt\')`'
#         )
#         rouge = None
#         return None, None, None

#     scores_rouge1 = [score['rouge-1']['r'] for score in scores]
#     scores_rouge2 = [score['rouge-2']['r'] for score in scores]
#     scores_rougeL = [score['rouge-l']['r'] for score in scores]
#     return max(scores_rouge1), max(scores_rouge2), max(scores_rougeL)

In [ ]:
# scores = rouge.compute(
#     predictions=[generated_summary], references=[reference_summary]
# )
# scores

In [ ]:
rouge = evaluate.load('rouge')

predictions = ["hello there 2", "general kenobi"]
references = ["hello there", "general kenobi"]
results = rouge.compute(predictions=predictions, references=references)
print(results)

In [ ]:
# def compute_rouge(eval_pred):    
#     print('eval_pred 합니당')
#     print(eval_pred)
#     predictions,references = eval_pred
    
#     return rouge.compute(predictions=predictions, references = references)

In [ ]:
!pip install datasets

In [13]:
import datasets
import numpy as np

In [16]:

rouge = datasets.load_metric("rouge")

from math import log
from numpy import array
from numpy import argmax

# def beam_search_decoder(data, k):
#     sequences = [[list(), 0.0]]
#     # walk over each step in sequence
#     for row in data:
#         all_candidates = list()
#         # expand each current candidate
#         for i in range(len(sequences)):
#             seq, score = sequences[i]
#             for j in range(len(row)):
#                 candidate = [seq + [j], score - log(row[j])]
#                 all_candidates.append(candidate)
#         # order all candidates by score
#         ordered = sorted(all_candidates, key=lambda tup:tup[1])
#         # select k best
#         sequences = ordered[:k]
#     return sequences


def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # np.where(a < 0)

    # print(labels_ids.shape, pred_ids.shape, type(labels_ids), type(pred_ids), sep = '\n\n')

    # all unnecessary tokens are removed
    
    # beam search
    # result = []
    # for i in range(pred_ids.shape[0]):
    #     result.append(beam_search_decoder(pred_ids[i], 3))

    # print(result)

  


    pred_str = tokenizer_gpt3.batch_decode(np.argmax(pred_ids, axis =2), skip_special_tokens=True)




    # labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer_gpt3.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),}

In [ ]:
labels_ids = np.arange(6).reshape(2,3)

In [ ]:
pred_ids = np.arange(24).reshape(2,3,4)
pred_ids

In [ ]:
print(np.argmax(pred_ids, axis =2)) #  greedy

* max_steps = 3

! 조정

In [67]:
# Trainer을 활용하는 형태로 모델 재생성
from transformers import Trainer, TrainingArguments
output_dir = '/content/drive/MyDrive/Jiwon/transformers'
metric_name = 'rouge'

training_arguments = TrainingArguments(
    output_dir, # output이 저장될 경로
    evaluation_strategy="epoch", #evaluation하는 빈도
    learning_rate = 2e-5, #learning_rate
    per_device_train_batch_size = 16, # 각 device 당 batch size
    per_device_eval_batch_size = 16, # evaluation 시에 batch size
    num_train_epochs = 5, # train 시킬 총 epochs
    weight_decay = 0.01, # weight decay
    max_steps = 500,
    save_total_limit=5,
    save_steps=500,
    # load_best_model_at_end = True
    # prediction_step = 100000,
    # do_eval = False
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
# from datasets import load_metric, Dataset
# # metric = load_metric('glue', 'mnli')

# def compute_metrics(eval_pred):    
#     predictions,labels = eval_pred
#     predictions = np.argmax(predictions, axis=1)
#     return metric.compute(predictions=predictions, references = labels)

In [63]:
val = Dataset.from_dict(encoded_data['test'][:320])

 ! 조정

In [68]:
trainer = Trainer(
    model=model_gpt3,                           # 학습시킬 model
    args=training_arguments,                  # TrainingArguments을 통해 설정한 arguments
    train_dataset=encoded_data['train'],    # training dataset
    eval_dataset=val,       # evaluation dataset
    compute_metrics=compute_metrics,
)

max_steps is given, it will override any value given in num_train_epochs


In [105]:
import gc
torch.cuda.empty_cache()
# del encoded_data
gc.collect()

8

In [106]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Sat Nov 12 07:16:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.172.01   Driver Version: 450.172.01   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/

In [75]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: target, input. If target, input are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 15250
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 500
  Number of trainable parameters = 118884864


RuntimeError: CUDA out of memory. Tried to allocate 24.00 MiB (GPU 0; 14.76 GiB total capacity; 13.21 GiB already allocated; 25.75 MiB free; 13.46 GiB reserved in total by PyTorch)

In [ ]:
# trainer.evaluate(encoded_dataset['test'])

In [ ]:
compute_metrics(encoded_data["test"])

In [38]:
# text = '근육이 커지기 위해서는'
input_ids = tokenizer_gpt3.encode(text, return_tensors='pt')

model_gpt3.to('cpu')
gen_ids = model_gpt3.generate(input_ids,
                          max_length=512,
                          repetition_penalty=2.0,
                          pad_token_id=tokenizer_gpt3.pad_token_id,
                          eos_token_id=tokenizer_gpt3.eos_token_id,
                          bos_token_id=tokenizer_gpt3.bos_token_id,
                          use_cache=True)
generated = tokenizer_gpt3.decode(gen_ids[0])
print(generated)

[CLS] 4일 오후 4시 충남 아산시 신리초등학교 체육관 2층 입구. 손 소독과 발열 체크를 한 뒤 체육관 안으로 들어가라는 안내문이 붙었다. 마스크를 착용하지 않으면 출입이 통제됐다. 체육관으로 들어가자 왼쪽에 설치된 대형 칠판에 [UNK] 사람 두 팔 간격 건강 거리 유지 [UNK] 라는 글이 크게 쓰여 있었다. 지난 3일 충남 아산지역에 폭우가 내리면서 온양천이 범람하자 신동 [UNK] 모종동 주민들이 급히 신리초로 대피했다. 신리초는 홍수와 지진 등 각종 재난사고 때 임시 주거시설로 지정된 곳이다. 3일 오후 5시까지 60여 명 수준이던 이재민은 오후 7시가 지나서는 100여 명을 넘어섰다. 아산시는 주민들이 한꺼번에 몰리자 수해와 더불어 신종 코로나바이러스 감염증 ( 코로나19 ) 이 확산할 것을 우려했다. 가장 먼저 주민들에게 마스크 착용을 의무화했다. 마스크를 쓰지 않으면 체육관 출입을 제한하고 체육관에 머무는 시간에도 밥을 먹거나 물을 마실 때를 제외하고 마스크를 벗지 못하게 했다. 이재민들은 집이나 농경지로 오갈 때도 출입 횟수에 관계없이 매번 발열 체크를 했다. 외부에서 다른 사람과 접촉할 가능성이 있어서다. 4일 오후 4시 30분쯤 체육관 안에는 주민 5명과 아산시청 공무원 [UNK] 자원봉사자 10여 명이 머물고 있었다. 주민들은 아산시가 제공한 매트리스 위에서 쉬거나 대화를 나눴다. 매트리스 1개마다 1명씩 배정됐다. 매트리스 간격은 2 ~ 3m가량으로 1개의 매트리스에 2명 이상이 앉지 못하도록 했다. 주민 간 접촉을 최소화하기 위해서다. 다른 대피소와 달리 신리초 체육관에는 소형 천막 ( 텐트 ) 이 지급되지 않았다. 이 때문에 충남도와 아산시 등 방역 당국이 바짝 긴장했다. 이곳에 머무르는 주민들은 대부분 70 ~ 80대 고령자로 코로나19에 노출될 경우 감염 위험이 높아서다. 아산시는 주민들이 식사를 할 때 밀접 접촉할 가능성이 크다고 판단, 모든 식사를 도시락으로 대체했다. 한 곳에 도시락을 쌓아 놓고 주민들이 줄을 서서 받지

In [53]:
generated.split(tokenizer_gpt3.sep_token)[-1:]

[' [PAD]퓘촷퇏 뙑 [unused1700] [unused280]썖 [unused392] [unused824]쉕톽 럸 [unused864] [unused1511] [unused634] [unused1408] [unused351] [unused772] [unused573] 렊 [unused349] [unused1962] [unused1625] [unused505] [unused147] [unused138] [unused1058] [unused779] [unused1187] 쉕 [unused1271] [unused1053] [unused1923]']

'[SEP]'

In [42]:
text = encoded_data['train']['input'][0] + encoded_data['train']['target'][0]
text

'4일 오후 4시 충남 아산시 신리초등학교 체육관 2층 입구.\n  손 소독과 발열 체크를 한 뒤 체육관 안으로 들어가라는 안내문이 붙었다.\n  마스크를 착용하지 않으면 출입이 통제됐다.\n  체육관으로 들어가자 왼쪽에 설치된 대형 칠판에 ‘사람 두 팔 간격 건강 거리 유지’라는 글이 크게 쓰여 있었다.\n  지난 3일 충남 아산지역에 폭우가 내리면서 온양천이 범람하자 신동·모종동 주민들이 급히 신리초로 대피했다.\n  신리초는 홍수와 지진 등 각종 재난사고 때 임시 주거시설로 지정된 곳이다.\n  3일 오후 5시까지 60여 명 수준이던 이재민은 오후 7시가 지나서는 100여 명을 넘어섰다.\n  아산시는 주민들이 한꺼번에 몰리자 수해와 더불어 신종 코로나바이러스 감염증(코로나19)이 확산할 것을 우려했다.\n  가장 먼저 주민들에게 마스크 착용을 의무화했다.\n  마스크를 쓰지 않으면 체육관 출입을 제한하고 체육관에 머무는 시간에도 밥을 먹거나 물을 마실 때를 제외하고 마스크를 벗지 못하게 했다.\n  이재민들은 집이나 농경지로 오갈 때도 출입 횟수에 관계없이 매번 발열 체크를 했다.\n  외부에서 다른 사람과 접촉할 가능성이 있어서다.\n  4일 오후 4시 30분쯤 체육관 안에는 주민 5명과 아산시청 공무원·자원봉사자 10여 명이 머물고 있었다.\n  주민들은 아산시가 제공한 매트리스 위에서 쉬거나 대화를 나눴다.\n  매트리스 1개마다 1명씩 배정됐다.\n  매트리스 간격은 2~3m가량으로 1개의 매트리스에 2명 이상이 앉지 못하도록 했다.\n  주민 간 접촉을 최소화하기 위해서다.\n  다른 대피소와 달리 신리초 체육관에는 소형 천막(텐트)이 지급되지 않았다.\n  이 때문에 충남도와 아산시 등 방역 당국이 바짝 긴장했다.\n  이곳에 머무르는 주민들은 대부분 70~80대 고령자로 코로나19에 노출될 경우 감염 위험이 높아서다.\n  아산시는 주민들이 식사를 할 때 밀접 접촉할 가능성이 크다고 판단, 모든 식사를 도시락으로 대체했다.\n  한 곳에 도

In [36]:
encoded_data['train']['target'][0]

'충남 아산지역에 폭우가 내려 온양천이 범람하자 주민들은 신리초로 대피했고 이에 아산시는 코로나19 확산이 걱정되어 마스크를 쓰지 않으면 출입을 제한했다. '

In [35]:
encoded_data['train']

Dataset({
    features: ['input', 'target', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 15250
})

In [109]:
torch.cuda.device_count()

1

## (4)Generate_Sample

* train_ptuning
* KoGPTConditionalGeneration

In [ ]:
! pip install --upgrade pip

In [ ]:
! git clone https://github.com/seujung/KoGPT2-summarization.git

In [ ]:
# ! pip install train_ptuning

In [ ]:
# ! pip install utils

In [ ]:
%cd KoGPT2-summarization

In [ ]:
import os
os.chdir('KoGPT2-summarization')

In [ ]:
! pip install torch==1.8.1
! pip install transformers==4.5.1
! pip install pytorch-lightning==1.1.0
! pip install streamlit==0.72.0

In [ ]:
!pip list | grep pytorch

In [ ]:
!pip install pytorch_lightning
! pip install transformers

GPT2 generator > summmary 

In [ ]:

import yaml
import torch
import torch.nn.functional as F
import utils
from train_ptuning import KoGPTConditionalGeneration
from utils import generate_next_token

In [ ]:
!mkdir log
!touch log/hparams.yaml

In [ ]:
hparams_file = 'log/hparams.yaml'
with open(hparams_file) as f:
    hparams = yaml.load(f, yaml.FullLoader)

In [ ]:
!touch log/KoGPT2_summary-last.ckpt

In [ ]:
inf = KoGPTConditionalGeneration.load_from_checkpoint('./log/KoGPT2_summary-last.ckpt', hparams=hparams)

In [ ]:
tokenizer = inf.tokenizer

In [ ]:
SUMMARY = ''
PTUNING = ''
EOS = ''

In [ ]:
text = """수도권 지역 일일 코로나19 신규 확진자가 지난 3차 유행 이후 반년 만에 연일 600명대를 기록하면서 다음 주 수도권 지역에 새로운 사회적 거리두기 체계를 적용할 수 있을지 관심이 쏠리고 있다.

젊은 층을 중심으로 확진자가 늘고 있고 인도에서 유래한 델타 변이 바이러스가 수도권 집단감염 사례에서 발견되는 등 유행 상황이 심상치 않다.

감염병 전문가들은 상황이 나아질 때까지 새 거리두기 적용을 미뤄야 한다고 조언했다. 새 거리두기를 시행한다면 단계를 높여서 적용하거나 여름 휴가, 변이 유행 등을 대비해 방역 조처를 강화해야 한다고 강조했다.

2일 질병관리청 중앙방역대책본부에 따르면 이날 0시 기준 국내 일일 코로나19 신규 확진자는 826명이다.

지난달 30일과 이달 1일 연이틀 700명대로 집계된 데 이어 이날 800명대로 급증했다. 이는 3차 유행이 정점에 도달한 직후였던 1월7일 869명 이후 176일 만에 최대 규모다. 국내 발생 확진자 수도 같은 기간 최대 규모인 765명을 기록했다. 일주일간 하루 평균 환자 수는 635.6명으로, 이틀 연속 600명대에서 증가하고 있다.

특히 수도권 지역 유행세가 크게 증가하고 있다. 수도권 지역 국내 발생 확진자는 지난달 30일 631명을 기록한 데 이어 이달 1일 607명, 이날 619명으로 집계됐다. 연일 600명대를 기록한 건 지난 1월7일 이후 처음이다.

최근 수도권 지역에서 유행 증가세를 보이자 중앙재난안전대책본부(중대본)와 수도권 3개 시·도는 당초 1일 적용하려던 새 거리두기 체계 시행을 일주일 연기했다. 중대본과 수도권 지자체는 이번 주까지 유행 상황을 지켜본 후 다음 주 초에 논의를 거쳐 새 체계 적용 여부를 결정할 계획이다.
"""

In [ ]:
tokenizer.encode(EOS)

In [ ]:
text = text.replace('\n', '')
input_tokens = tokenizer.encode(PTUNING)* 10 + tokenizer.encode(text) + tokenizer.encode(SUMMARY)
input_tensor = torch.tensor(input_tokens).unsqueeze(0)

eos_id = tokenizer.encode(EOS)[0]

In [ ]:
while True:
    pred = inf.model(input_tensor)
    next_token = generate_next_token(pred.logits, temperature=1.0, top_p=0.8)

    if next_token.item() == eos_id:
        break
    else:
        input_tensor = torch.cat([input_tensor, next_token.unsqueeze(0)],1)

In [ ]:
tokenizer.decode(input_tensor[0])

In [ ]:
tokenizer.decode(input_tensor[0]).split('')[-1].strip()

# Daily Report 

## DAY02

> Task1) Preprocessing


> Task2) Checkpoint
- [checkpoint](https://dacon.io/en/competitions/official/235875/codeshare/4520)
 * output_dir 설정 
 * save_total_limit
 * save_stepts
 * load_best_model_at_end = True
 * resum_from_checkpoint
- [resume_from_checkpoint](https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments)
 * Q1)`Resuming training from a checkpoint` can be done when calling Trainer.train() with either:
   * `resume_from_checkpoint=True` which will resume training from the latest checkpoint
   * `resume_from_checkpoint=checkpoint_dir` which will resume training from the specific checkpoint in the directory passed.

> Task) Generate_Summary


In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs, labels=inputs["input_ids"])
loss = outputs.loss
logits = outputs.logits
logits

In [ ]:
import numpy as np

pred_str = tokenizer.batch_decode(np.argmax(logits.detach().numpy(), axis =2), skip_special_tokens=True)
pred_str

In [ ]:
outputs = model(inputs)
logits = outputs.logits
logits